In [15]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    OmegaConf.resolve(cfg)

In [16]:
cfg

{'optimizer': {'lr': 0.008}, 'lr_scheduler': {'linear_lr_warmup': {'start_factor': 0.25, 'end_factor': 1.0, 'total_iters': 20}, 'cosine_annealing_warm_restarts': {'T_0': 40, 'T_mult': 1, 'eta_min': 1e-05}}, 'train_epochs': 500, 'num_parallel_environments': 200, 'device': 'cuda', 'random_seed': 0, 'experiment_name': 'standing_up', 'xla_gpu_memory_fraction': 0.6, 'environment': {'num_parallel_environments': 200, 'sim_frames_per_step': 5, 'mujoco_timestep': 0.005, 'model_path': './external/unitree_mj_models/go2/scene.xml', 'initial_noise_scale': 0.1, 'observation_size': 37, 'action_size': 12}, 'experiment': {'body_name': 'base_link', 'body_angle_reward_scale': 1.0, 'body_height_reward_scale': 1.0, 'energy_reward_scale': 1.0, 'distance_from_origin_reward_scale': 1.0, 'joint_limit_reward_scale': 1.0}, 'agent': {'observation_size': 37, 'action_size': 12, 'train_sequence_length': 200, 'network_hidden_size': 16, 'num_hidden_layers': 0, 'lambda_': 0.99, 'epsilon': 0.3, 'discounting': 0.97, 'rew

In [29]:
from unitree_robot.common.environments import MujocoMjxEnv

environment = MujocoMjxEnv(**cfg.environment)
environment.mjx_data_initial.__dir__()

['time',
 'qpos',
 'qvel',
 'act',
 'qacc_warmstart',
 'plugin_state',
 'ctrl',
 'qfrc_applied',
 'xfrc_applied',
 'eq_active',
 'mocap_pos',
 'mocap_quat',
 'qacc',
 'act_dot',
 'userdata',
 'sensordata',
 'xpos',
 'xquat',
 'xmat',
 'xipos',
 'ximat',
 'xanchor',
 'xaxis',
 'ten_length',
 'geom_xpos',
 'geom_xmat',
 'site_xpos',
 'site_xmat',
 'cam_xpos',
 'cam_xmat',
 'subtree_com',
 'cvel',
 'qfrc_bias',
 'qfrc_gravcomp',
 'qfrc_fluid',
 'qfrc_passive',
 'qfrc_actuator',
 'actuator_force',
 'qfrc_smooth',
 'qacc_smooth',
 'qfrc_constraint',
 'qfrc_inverse',
 '_impl',
 '__module__',
 '__firstlineno__',
 '__annotations__',
 '__doc__',
 'impl',
 '__getattr__',
 '__getitem__',
 '__static_attributes__',
 '__dataclass_params__',
 '__dataclass_fields__',
 '__replace__',
 '__hash__',
 '__init__',
 '__repr__',
 '__eq__',
 '__setattr__',
 '__delattr__',
 '__match_args__',
 'replace',
 'bind',
 '__init_subclass__',
 'fields',
 'tree_replace',
 '__dict__',
 '__weakref__',
 '__new__',
 '__str__

In [81]:
actuator_map = tuple(zip(
    range(environment.mjx_model.nu),
    environment.mjx_model.name_actuatoradr,
    [environment.mjx_model.names[i:].split(b'\x00')[0] for i in environment.mjx_model.name_actuatoradr]))

actuator_map


((0, np.int32(591), b'FR_hip'),
 (1, np.int32(598), b'FR_thigh'),
 (2, np.int32(607), b'FR_calf'),
 (3, np.int32(615), b'FL_hip'),
 (4, np.int32(622), b'FL_thigh'),
 (5, np.int32(631), b'FL_calf'),
 (6, np.int32(639), b'RR_hip'),
 (7, np.int32(646), b'RR_thigh'),
 (8, np.int32(655), b'RR_calf'),
 (9, np.int32(663), b'RL_hip'),
 (10, np.int32(670), b'RL_thigh'),
 (11, np.int32(679), b'RL_calf'))